### NX-421: Neural signals and signal processing
# Miniproject 1 - Variant 3

### Camille Dorster, Toufan Kashaev, Johan Bordet

### Importing the relevant libraries

In [1]:
import sys
import os

#####################
# Import of utils.py functions
#####################
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist, interactive_MCQ

####################
# DIPY_HOME should be set prior to import of dipy to make sure all downloads point to the right folder
####################
os.environ["DIPY_HOME"] = "/home/jovyan/data"

#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

import fsl.wrappers
from fsl.wrappers import fslmaths

import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri

import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

import ants

import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt

# General purpose imports to handle paths, files etc
import glob
import pandas as pd
import numpy as np
import json
import subprocess

# Utility function used to load json from file name
def get_json_from_file(fname):
    f = open(fname)
    data = json.load(f)
    f.close()
    return data

In [2]:
%gui wx

Gtk-Message: 09:53:44.852: Failed to load module "canberra-gtk-module"


### Starting FSLeyes

In [3]:
################
# Start FSLeyes
################
fsleyesDisplay = FSLeyesServer()
fsleyesDisplay.show()

09:53:46: Debug: Adding duplicate image handler for 'Windows bitmap file'
09:53:46: Debug: Adding duplicate animation handler for '1' type
09:53:46: Debug: Adding duplicate animation handler for '2' type
09:53:46: Debug: Adding duplicate image handler for 'Windows bitmap file'
09:53:46: Debug: Adding duplicate animation handler for '1' type
09:53:46: Debug: Adding duplicate animation handler for '2' type

(ipykernel_launcher.py:5706): Gtk-CRITICAL **: 09:53:46.977: gtk_window_resize: assertion 'height > 0' failed


### Preparing the paths and structure of the data in folders

In [13]:
dataset_id = 'mydataset'
subject_id = '101410' 
subject_dir = 'sub-{}'.format(subject_id)

bids_root = op.join(sample_path, dataset_id)
deriv_root = op.join(bids_root, 'derivatives')
preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')
preproc_subject_path = op.join(preproc_root, subject_dir)
preproc_anat_path = op.join(preproc_root, subject_dir, 'anat')
preproc_func_path = op.join(preproc_root, subject_dir, 'func')
preproc_fmap_path = op.join(preproc_root, subject_dir, 'fmap')

###################
# Create folders relevant for preprocessing.
###################
sample_path = "/home/jovyan/data"
mkdir_no_exist(sample_path)
mkdir_no_exist(bids_root)
mkdir_no_exist(deriv_root)
mkdir_no_exist(preproc_root)
mkdir_no_exist(preproc_subject_path)
mkdir_no_exist(preproc_anat_path)
mkdir_no_exist(preproc_func_path)
mkdir_no_exist(preproc_fmap_path)

## Step 1 : skull stripping

### Applying BET to get the betted brain mask

In [17]:
resulting_mask_path = op.join(preproc_anat_path, 'sub-{}_T1w_mask'.format(subject_id))
anatomical_path = op.join(bids_root, subject_dir, 'anat', 'sub-{}_T1w.nii.gz'.format(subject_id))
betted_brain_path = op.join(preproc_anat_path, 'sub-{}_T1w'.format(subject_id))

os.system('bet {} {} -m {}'.format(anatomical_path, betted_brain_path, '-R')) # Robust parameter used

print("Done with BET.")

Done with BET.


In [18]:
# Display the mask
fsleyesDisplay.load(resulting_mask_path)

### Applying the betted brain mask to the original brain

In [11]:
anatomical_path = op.join(bids_root, subject_dir, 'anat', 'sub-{}_T1w.nii.gz'.format(subject_id)) # The original brain
betted_brain_path = op.join(preproc_anat_path, 'sub-{}_T1w.nii.gz'.format(subject_id)) # The brain without skull is in the derivatives folder
resulting_mask_path = op.join(preproc_anat_path, 'sub-{}_T1w_mask'.format(subject_id)) # The mask to use

os.system('fslmaths {} -mas {} {}'.format(anatomical_path, resulting_mask_path, betted_brain_path))

print("Mask applied.")

0

In [19]:
# Display the betted brain
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(betted_brain_path)

## Step 2 : tissue segmentation

In [ ]:
fast_target = betted_brain_path

[os.remove(f) for f in glob.glob(op.join(preproc_anat_path, '*fast*'))] # Just to clean the directory in between runs of the cell
segmentation_path = op.join(preproc_anat_path, 'sub-101410_T1w_fast')
fast(imgs=[fast_target], out=segmentation_path, n_classes=3)

In [10]:
# Display the segmented brain
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(bet_path)
fsleyesDisplay.load(glob.glob(op.join(preproc_anat_path,'*pve_0*'))[0])
fsleyesDisplay.load(glob.glob(op.join(preproc_anat_path,'*pve_1*'))[0])
fsleyesDisplay.load(glob.glob(op.join(preproc_anat_path,'*pve_2*'))[0])
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[1]).cmap = 'Red'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[2]).cmap = 'Green'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[3]).cmap = 'Blue'

## Step 3 : Variance normalization and concatenation of the runs

In [ ]:
# TBD

## Step 4 : motion correction

In [ ]:
from fsl.wrappers import mcflirt

path_original_data = op.join(bids_root, 'sub-101410', 'func', 'sub-101410_task-sitrep_run-01_bold')
path_moco_data = op.join(preproc_root, 'sub-101410', 'func', 'sub-101410_task-sitrep_run-01_bold_moco')
mcflirt(infile=path_original_data,o=path_moco_data, plots=True, report=True, dof=6, mats=True)

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(path_original_data)
fsleyesDisplay.load(path_moco_data)